# Options in LaboneQ Applications Library

Options provided for workflow can be created via "options" classes.

There are two categories of options:

* **TaskOptions**:
    These options are used to set parameters for tasks in the workflow. An example of `TaskOptions` class is the `BaseExperimentOptions` which contains options for setting how experiments should run. For examples, the number of experiment shots `counts`, the averaging mode `averaging_mode`, the acquisition type `acquisition_type` and so on. 

* **WorkflowOptions**:
    These options are used to set parameters for the workflow. These include the settings for operating the workflow, such as `logbook` which specifies the protocol for storing a collection of records of workflow execution.
    In addition, options for the constituent tasks of the workflow can also be set by specifying the task name and the options for that task.



In [ ]:
from __future__ import annotations

from laboneq_applications import workflow
from laboneq_applications.experiments.options import (
    BaseExperimentOptions,
    TuneupExperimentOptions,
)

# Create a new experiment options class
The options for an experiment can be created by inheriting from `BaseExperimentOptions` class.

The options class attributes must have default value. This way, options can be used without specifying them, and the default values will be used.

In [ ]:
class NewExperimentOptions(BaseExperimentOptions):
    operand: int = 1

In [ ]:
opt = NewExperimentOptions()

In [ ]:
opt

If the options classes are defined without default values, an error will be raised when the options instance is created.

In [ ]:
class InvalidExperimentOptions(BaseExperimentOptions):
    operand: int


opt = InvalidExperimentOptions(operand=1)

# Standard experiment options classes

The library provides a few standard options classes, such as `TuneupExperimentOptions`, which can be used for typical tune-up experiments or as a starting point for creating new options classes for more advanced experiments.

In [ ]:
opt = TuneupExperimentOptions()
opt

# Enable the options feature in the workflow

The advantage of using the options feature of a workflow is the automatic passing of options to the tasks and other features like run_until, etc.

At the moment, the options feature is enabled by default in the workflow when the arguments include `options`. In addition, the types for `options` must conform to a certain rule.

Let's illustrate the concepts via an example. 

`myworkflow` contains task `mytask` which takes in an argument and options. 

To use `myworkflow` with options, we must provide the right types for the `options` argument in `myworkflow`.

The supported types are:
- `WorkflowOptionsA | None = None`
- `Union[WorkflowOptionsA, None] = None`
- `Optional[WorkflowOptionsA]`

where `WorkflowOptionsA` is a subclass of `WorkflowOptions`.

From Python 3.10 onward, it is recommended to use `WorkflowOptionsA | None` to conform with the standard practice.

On Python 3.9, `from _future_ import annotations` must be imported to use `WorkflowOptionsA | None`.


In [ ]:
class WorkflowOptionsA(workflow.WorkflowOptions):
    count : int = 1024

In [ ]:
@workflow.task
def add(x, options: NewExperimentOptions|None = None):
    return x + options.operand


@workflow.task
def multiply(x, options: NewExperimentOptions| None = None):
    return x * options.operand

In [ ]:
@workflow.workflow
def myworkflow(options: WorkflowOptionsA | None = None):
    add(x=1)
    multiply(x=2)

# Disallowed types for options

When the type provided for the options includes a subclass of `WorkflowOptions`, we assume that users are attempting to use the options feature. Hence, if the specified type does not follow the form of the above-mentioned types, an error will be raised to inform the user about this, as shown below. 

In [ ]:
# an error will be raised


@workflow.workflow
def invalid_workflow(options: WorkflowOptionsA | str):
    add(x=1)
    multiply(x=2)

# Run the workflow with options

We can create the options for a specific workflow directly by calling the `options()` method on the workflow. This returns an `OptionBuilder` instance, a wrapper of the `WorkflowOption` instance that contains the options for this specific workflow.

`OptionBuilder` helps to set the options fields more easily.

In [ ]:
workflow_opt = myworkflow.options()

# Querying and setting the option fields

The fields of the workflow options together with the options of nested tasks and workflows are visible as attributes of the `OptionBuilder` instance.

In [ ]:
dir(workflow_opt)

To see the values of an option field, simply access it as an attribute, as shown below for the case of field `count`. Here, we see that there are three `count` fields shown together with its values: one at the top-level of the options `(base,1024)` and two others at the task levels `add` and `multiply`

In [ ]:
workflow_opt.count

To set all the fields `count`

In [ ]:
workflow_opt.count(2048)
workflow_opt.count

Standard array slicing is also supported

In [ ]:
workflow_opt.count[1:](1024)
workflow_opt.count

We can inspect the options of specific task of the workflow by accessing the `base` attribute of the `OptionBuilder`, which is simply the original `WorkflowOptions` object.

In [ ]:
workflow_opt.base.task_options["add"]

To set the value of field `count` for task `add`, 

In [ ]:
workflow_opt.base.task_options["add"].count = 512

Now to run the workflow with the modified options, simply pass the options to the workflow initialization

In [ ]:
res = myworkflow(options=workflow_opt).run()
res.tasks[1].output